Environment: This script should be run with the `r_environment_pathogen_atlas` environment using the devcontainer `devcontainer_R` 

In [ ]:
library(Seurat)
library(ggplot2)
library(patchwork)
library(Matrix)


set.seed(0)

Read in snRNA data from all replicates

In [ ]:
data_folder <- "../../data"

In [ ]:
seq_folder = file.path(data_folder, 'AvrRpt2_alone2.rds')

In [ ]:
scrna = readRDS(seq_folder)

Add pseudotime to the snRNA data

In [ ]:
rep1_folder = file.path(data_folder, 'AvrRpt2_alone.rds')
rep1 = readRDS(rep1_folder)
rep1_pseudotime = read.csv(file.path(data_folder, 'temp_objects', 'pseudotime_csv.csv'))
rep1@meta.data$index <- rownames(rep1@meta.data)
cleaned_vector <- gsub("col_|_rep1", "", rownames(scrna@meta.data))
copyscrna <- scrna@meta.data
rownames(copyscrna) <- cleaned_vector

Formatting the snRNA

In [ ]:
rep1@meta.data <- merge(rep1@meta.data, copyscrna[c("sub_clst_rna", 'celltype', 'SCT_snn_res.1')], by = 0, all.x = TRUE)

rownames(rep1@meta.data) <- rep1@meta.data$Row.names

Idents(object = rep1) <- "celltype.y"

rep1 <- subset(rep1, idents = c('Vasculature', 'Mesophyll', 'Epidermis', 'Unknown'))

merged_df <- merge(rep1@meta.data, rep1_pseudotime, by = 'index', all.x = TRUE)
rep1@meta.data <- merged_df

rownames(rep1@meta.data) <- rep1@meta.data$index

DefaultAssay(rep1) <- "SCT"

rep1@meta.data$celltype <- ifelse(rep1@meta.data$celltype.y == "", "Unknown", rep1@meta.data$celltype.y)
rep1 <- subset(x = rep1, subset = celltype.y != c('Unknown'))

Reading in the DC3000 snRNA data

In [ ]:
dc3000 <- readRDS(file.path(data_folder, 'DC3000_alone.rds'))

dc3000 <- subset(dc3000, subset = sample.order == '01_DC3000_24h')


Finding anchors between the other snRNA datasets and DC3000

In [ ]:
anchors <- list()

In [ ]:
anchors[[1]] <- FindTransferAnchors(
    reference = rep1,
    query = dc3000,
    reference.reduction = "pca", 
    normalization.method = "SCT"
)

In [ ]:

# Replace NaN values with 0 in the 'dpt_pseudotime' column
rep1@meta.data$dpt_pseudotime[is.na(rep1@meta.data$dpt_pseudotime)] <- 0

sample_vector <- rep1@meta.data$dpt_pseudotime 
sparse_matrix <- t(sparseMatrix(i = 1:length(sample_vector),
                              j = rep(1, length(sample_vector)),
                              x = sample_vector))
rep1[['pseudotime']] = t(sparse_matrix)
colnames(sparse_matrix) <- colnames(rep1)
rownames(sparse_matrix) <- c('pseudotime')

mapping dc3000 to the avr anchors

In [ ]:
dc3000 <- MapQuery(
    anchorset = anchors[[1]], 
    query = dc3000,
    reference = rep1, 
    refdata = list(pseudotime = sparse_matrix),
    reference.reduction = "pca",
)

Transferring pseudotime

In [ ]:
sparse_pseudo <- dc3000@assays$pseudotime@data
sparse_vector <- sparse_pseudo[1,]
dense_vector <- as.vector(sparse_vector)

In [ ]:
dc3000@meta.data$pseudotime <- dense_vector 

In [ ]:
dc3000@meta.data$pseudotime[is.na(dc3000@meta.data$pseudotime)] <- 0

Saving dc3000 snRNA with pseudotime

In [ ]:
dior::write_h5(dc3000, file= file.path(data_folder, "DC3000_alone.h5"), object.type = 'seurat')